# 🪰 Genetic Diversity and Population Structure of BSF  🪰 

This tutorial guides you through the complete workflow for analyzing genetic diversity and population structure in **Black Soldier Fly (BSF)** populations using whole-genome sequencing data.

🔔 **Note:** **Step 6 (Phasing and Imputation is optional)**. Include it when you have low-coverage whole-genome sequence data. In this case, I used the high-coverage SNPs (VCF) as a reference panel to impute the low-coverage data (sequence at 1x).

## 🧰 Setup Instructions

!conda create -n bsf_genomics fastqc bwa samtools bcftools gatk picard vcftools plink beagle r-quilt admixture treemix -c bioconda -c conda-forge \
!conda activate bsf_genomics

## 📁 Input Files
- Paired-end FASTQ files: [Project: PRJEB58720](https://www.ebi.ac.uk/ena/browser/view/PRJEB58720), [DOI](https://doi.org/10.1101/2023.10.21.563413)
- Reference genome: [BSF_reference.fasta](https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_905115235.1/)
- Sample metadata: `sample_pop.txt`

## Quality Control

In [ ]:
cd 2_fastqc

# Create soft links for all the files in 1_data
for fastq in ../Data/*.fastq.gz; do
  ln -s $fastq
done

# Run fastqc on the files
fastqc -t 4 *.fastq

## Mapping to Reference

In [ ]:
#!/bin/bash

#Index reference genome
bwa index reference_genome.fasta

# Path to the reference genome indexed with BWA
reference_genome="path/to/ref"

# Loop through each forward-read file
# Change the file extension if its not R1.fastq.gz || R2.fastq.gz 

for forward_file in ../Data/*.R1.fastq.gz; do
     # Get the base filename without the extension
     base_filename=$(basename -- "$forward_file" | sed 's/.R1.fq.gz//')

     # Generate the paths for the forward and reverse-read files
     reverse_file="../1_data/${base_filename}.R2.fq.gz"

     echo "Processing: ${base_filename}"

     # Perform the alignment using bwa mem, convert the sam to bam, and sort the bam
     # Change the read group according to read description and the sequencing platform used.
     bwa mem -M -t 8 -R "@RG\tID:${base_filename}\tPL:BGISEQ-500\tSM:${base_filename}" \
     "$reference_genome" "$forward_file" "$reverse_file" | \
     samtools view -bS | \
     samtools sort --output-fmt BAM -@ 8 -o "${base_filename}.sorted.bam" -

     # Index the sorted BAM file
     samtools index ${base_filename}.sorted.bam


    echo "Done Processing: ${base_filename}"

done

## Mark Duplicates

In [ ]:
#!/bin/bash

# Input directory containing BAM files
bam_directory="path/to/bamfiles"

# Output directory for processed BAM files
output_directory="./"

# Check if the input directory exists
if [ ! -d "$bam_directory" ]; then
    echo "Input directory $bam_directory does not exist"
    exit 1
fi

# Create the output directory if it doesn't exist
mkdir -p "$output_directory"

# Iterate over each .bam file in the input directory
for bam in "$bam_directory"*.bam; do
    if [ -f "$bam" ]; then
        base_name=$(basename "${bam%.*}")
        echo "Processing ${bam}"

        java -Xmx10G -jar picard.jar MarkDuplicates \
                    I="${bam}" \
                    O="${output_directory}${base_name}_markdup.bam" \
                    M="${output_directory}${base_name}_markdup_metrics.txt" \
                    OPTICAL_DUPLICATE_PIXEL_DISTANCE=100 \
                    CREATE_INDEX=true \
                    VALIDATION_STRINGENCY=LENIENT \
                    REMOVE_DUPLICATES=true \
                    ASSUME_SORT_ORDER=coordinate

      else
        echo "No .bam files found in $bam_directory"
        exit 1
    fi

done

echo "Completed marking duplicates"


## Variant Calling

In [ ]:
#!/bin/bash

#Running freebayes in parrallel
reference_genome="path/to/ref"

# Create a list with all your bam files
ls -l "path/to/markdup.bam" | awk '{print $NF}' | xargs -I{} readlink -f {} > bamlist

# Run freebayes
# Here the genome is chunked into regions of 100000 (1mb). 
# Ran on 20 processor (20 jobs will be ran in parallel).
# This can be adjusted based on the size of the genome and the computational resources

freebayes-parallel \
   <(fasta_generate_regions.py ${ref}.fai 100000) 20 \
   --fasta-reference ${reference_genome}  \
   --bam-list bamlist  > output.vcf

## VCF Filtering

In [ ]:
#!/bin/bash

#SNP filtering tutorial: https://ddocent.com/filtering/

bcftools index --threads 8 output.vcf.gz

#step 1: filters on high missignesss (>50%), minor allele count (mac3), and Quality (Q30)
vcftools --gzvcf output.vcf.gz  \
         --max-missing 0.5 \
         --mac 3 \
         --minQ 30 \
         --recode \
         --recode-INFO-all \
         --stdout | bgzip -c --threads 8 > output.mac3.Q30.vcf.gz

#step 2: Remove individuals with high missing data (> 50%)
vcftools --gzvcf output.mac3.Q30.vcf.gz --missing-indv
mawk '!/IN/' out.imiss | cut -f5 > totalmissing
mawk '$5 > 0.5' out.imiss | cut -f1 > lowDP.indv

vcftools --gzvcf output.mac3.Q30.vcf.gz \
         --remove lowDP.indv \
         --recode \
         --recode-INFO-all \
         --stdout | bgzip -c --threads 8 > output.mac3.Q30.lDP.vcf.gz

#Step 3: Decompose biallelic block substitutions
# More info on vt here: https://genome.sph.umich.edu/wiki/Vt#Decompose_biallelic_block_substitutions
bcftools index --threads 8 output.mac3.Q30.lDP.vcf.gz

vt decompose_blocksub output.mac3.Q30.lDP.vcf.gz | \
vt uniq - | \
bgzip -c --threads 8 > output.mac3.Q30.lDP.dec.dup.vcf.gz

bcftools index --threads 8 output.mac3.Q30.lDP.dec.dup.vcf.gz

#Step 4: Remove Indels and multiallelic (--min-alleles/--max-alleles 2),Kept only biallelic SNPs
bcftools view -m2 -M2 -v output.mac3.Q30.lDP.dec.dup.vcf.gz \
                     -Oz -o output.mac3.Q30.lDP.dec.dup.snps.vcf.gz --threads 8

bcftools index --threads 8 output.mac3.Q30.lDP.dec.dup.snps.vcf.gz

#Step 5: Filter on meanDP,adjust based on your data.
vcftools --gzvcf output.mac3.Q30.lDP.dec.dup.snps.vcf.gz \
         --min-meanDP 5 \
         --max-meanDP 25 \
         --recode \
         --recode-INFO-all \
         --stdout | bgzip -c --threads 8 > output.mac3.Q30.lDP.dec.dup.snps.mnDP.vcf.gz

bcftools index --threads 8 output.mac3.Q30.lDP.dec.dup.snps.mnDP.vcf.gz

#Step 6: Missigness, adjust based on your data and downstream analysis intended
vcftools --gzvcf output.mac3.Q30.lDP.dec.dup.snps.mnDP.vcf.gz \
         --max-missing 0.95 \
         --recode \
         --recode-INFO-all \
         --stdout | bgzip -c --threads 8 > output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.vcf.gz

bcftools index --threads 8 output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.vcf.gz

#Step 7: Pop-specific missigness (Based on the number of populations you have) (optional)
# Downdload the script here: https://www.ddocent.com/filtering/

bash pop_missing_filter.sh output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.vcf.gz popmap 0.1 3 output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop

bgzip --threads 8 output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop.recode.vcf

bcftools index --threads 8 output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop.recode.vcf.gz

#Step 8: Allelic balance, adjust based on your data
bcftools filter -i '(INFO/AB > 0.25 && INFO/AB < 0.75) || INFO/AB < 0.01' -Oz -o output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop.AB.vcf.gz \
                                                                                 output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop.recode.vcf.gz --threads 8

bcftools index --threads 8 output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop.AB.vcf.gz

#Step 9: Minor Allele Frequency (MAF) threshold to exclude monomorphic sites (which have a MAF of 0).
vcftools --gzvcf output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop.AB.vcf.gz \
         --maf 0.001 \
         --recode \
         --recode-INFO-all \
         --stdout | bgzip -c --threads 8 > output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop.AB.lmaf.vcf.gz


## Phasing (_optional_)

In [ ]:
#!/bin/bash

#Phasing one chromosome at a time
VCF='Chr1.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop.AB.lmaf.vcf.gz'  # Change this to your input VCF file path
Chr='Chr1'         # Change this to your desired chromosome prefix (e.g., Chr1)

#Step 1: Phase the VCF using Beagle
java -Xmx100g -jar beagle/beagle5.jar gt=${VCF} \
                   out=${Chr}_phased gp=true burnin=10 iterations=40 impute=false nthreads=40 chrom=${Chr}

#Step 2: Index the phased VCF output
bcftools index --threads 8 ${Chr}_phased.vcf.gz

## Imputation (_optional_)

In [ ]:
#!/bin/bash

#Imputation using QUILT v1.0.5: https://github.com/rwdavies/QUILT/blob/master/README_QUILT2.org
#Step 1: Building a reference panel

Chr="Chr1"

# Define paths
VCF_file="${Chr}_reference_panel.bcf"
chunks_file="chunks.${Chr}.txt" # dat <- QUILT::quilt_chunk_map("Chr1", "genetic_map.txt")
Out_dir="/Quilt_${Chr}/"

# Ensure output directory exists
mkdir -p "$Out_dir"

# Get the chunk from the array task ID
chunk_info=$(tail -n +2 "$chunks_file" | sed -n "$((SLURM_ARRAY_TASK_ID + 1))p")
chunk=$(echo "$chunk_info" | cut -f1)
chr=$(echo "$chunk_info" | cut -f2)
region=$(echo "$chunk_info" | cut -f3)

# Extract region start and end
regionStart=$(echo "$region" | cut -d":" -f2 | cut -d"-" -f1)
regionEnd=$(echo "$region" | cut -d"-" -f2)

# Check if output exists, skip if found
if [ ! -f "$Out_dir/RData/QUILT_prepared_reference.${Chr}.$regionStart.$regionEnd.RData" ]; then
  echo "Processing chunk $chunk for $Chr (region $regionStart-$regionEnd)"

  # Run QUILT2_prepare_reference.R
  QUILT2_prepare_reference.R \
  --outputdir="$Out_dir" \
  --chr="$Chr" \
  --nGen=100 \
  --regionStart="$regionStart" \
  --regionEnd="$regionEnd" \
  --buffer=500000 \
  --reference_vcf_file="$VCF_file"

else
  echo "Skipping chunk $chunk for $Chr as output already exists."
fi

################################################################################
#Step 2: Imputation
# Set the chromosome from the input argument
Chr=$1

set -e   # Exit immediately if a command exits with a non-zero status

echo "Job started at $(date '+%d_%m_%y_%H_%M_%S')"

# Define coverages and paths
coverages=("1x" )
num_coverages=${#coverages[@]}
chunks_file="/${Chr}/chunks.${Chr}.txt"

# Calculate chunk and coverage index based on the array task ID
chunk_index=$((SLURM_ARRAY_TASK_ID / num_coverages))
coverage_index=$((SLURM_ARRAY_TASK_ID % num_coverages))

# Read chunk and region based on chunk_index
chunk=$(awk "NR==$((chunk_index + 2))" "$chunks_file" | cut -f1)  # +2 to skip header
region=$(awk "NR==$((chunk_index + 2))" "$chunks_file" | cut -f3)  # Use 3rd column for region
Coverage=${coverages[$coverage_index]}

# Extract start and end positions from region
if [[ "$region" =~ ^[^:]+:([0-9]+)-([0-9]+)$ ]]; then
    regionStart="${BASH_REMATCH[1]}"
    regionEnd="${BASH_REMATCH[2]}"
else
    echo "Error: Invalid region format in $chunks_file on line $((chunk_index + 2))"
    exit 1
fi

# Debug output to verify values
echo "Chunk: $chunk, Region: $region, Start: $regionStart, End: $regionEnd, Coverage: $Coverage"

# Define paths (build reference panel, bamlist of samples to be imputed, and output directory)
Ref="/Quilt_${Chr}/RData/"
Bamlist="bamlist"
Out_dir="/${Chr}/Quilt/"

# Ensure output directory exists
mkdir -p "$Out_dir"

# Check if the output file already exists to avoid reprocessing
if [ ! -f "$Out_dir/quilt.${Chr}.$regionStart.$regionEnd.vcf.gz" ]; then
    echo "Processing $Chr, Coverage $Coverage, Chunk $chunk, Region ${regionStart}-${regionEnd}"

    # Run Quilt2 Imputation
    QUILT.R \
      --prepared_reference_filename="${Ref}QUILT_prepared_reference.${Chr}.${regionStart}.${regionEnd}.RData" \
      --bamlist="${Bamlist}" \
      --method=diploid \
      --chr="$Chr" \
      --regionStart="${regionStart}" \
      --regionEnd="${regionEnd}" \
      --buffer=500000 \
      --output_filename="${Out_dir}/quilt.${Chr}.${regionStart}.${regionEnd}.vcf.gz" \
      --nCores=1
else
    echo "Skipping $Chr, Coverage $Coverage, Chunk $chunk: Output already exists."
fi

echo "Job completed for ${Chr} at $(date '+%d_%m_%y_%H_%M_%S')"



## Population Structure Analysis
### PCA

In [ ]:
################################################################################
#                Genotypic data quality control and PCA analysis
################################################################################

# Use PLINK to pre-process (i.e. quality control) of BSF data. 
# Perform PCA analysis, and then make PCA plot using ggplot2 

####################
# Lets get started 
###################

# We will use genotypic data in VCF format.
# Plink options
setwd("/PCA/")
#system("plink-1.9-rc --help") # to get more info on plink options

################################################################################
#  For detailed information about options visit link below
#                           https://www.cog-genomics.org/plink/1.9/data
################################################################################

# lets start with VCF and recode into plink format for down stream analysis

system("plink-1.9-rc --vcf output.mac3.Q30.lDP.dec.dup.snps.mnDP.miss0.95.pop.AB.lmaf.vcf.gz --recode --keep-allele-order --nonfounders --allow-no-sex --double-id --out output")

#list.files("./")

# lets make .bed formated plink file
system("plink-1.9-rc --file output --make-bed --nonfounders --allow-no-sex --keep-allele-order --double-id --out output_bed")

# QC options 

# 1. Missingness per SNP: --geno  
# 2. Missingness per individual: --mind 
# 3. Minor allele frequency: --maf

#20% mSNP, 20% mind, maf 5%
system("plink-1.9-rc --bfile output_bed --geno 0.2 --mind 0.2 --maf 0.05 --make-bed --keep-allele-order --double-id --out output_qc_out")

# lets make vcf file
system("plink-1.9-rc --bfile output_qc_out --recode vcf-iid --double-id --keep-allele-order --out output_qc_out_vcf")

# Now, we have a fully filtered VCF we can start some analyses with it.
# First of all we will investigate population genetic structure using principal component analysis (PCA).
#
# PCA analysis: a model-free method -------------------------------------------------------------

# one of the assumptions of PCA analysis is that SNP data we use is independent (i.e., there are no spurious correlation among measured variables)
# this is not true for most of SNP dataset as allele frequencies are correlated due to physical linkage and linkage disequilibrium .

# step 1 identify prune sites
# window of 50 Kb
# window step size 10
# r2 threshold 0.2

system(
  "plink-1.9-rc --vcf output_qc_out_vcf.vcf --double-id --allow-extra-chr --set-missing-var-ids @:# --recode vcf-iid --indep-pairwise 50 10 0.2 --out output_PRUNED"
)

# step 2: PCA analysis
system(
  "plink-1.9-rc --vcf output_qc_out_vcf.vcf --double-id --allow-extra-chr --set-missing-var-ids @:# --extract output_PRUNED.prune.in --pca --make-bed --out output_prune_pca"
)

# pca plot
library(tidyverse)

plinkPCA <- read_table("output_prune_pca.eigenvec", col_names = F)
plinkPCA <- plinkPCA[,c(-1,-2)] # remove first two columns
EigenValue <- scan("output_prune_pca.eigenval")
#view(EigenValue)

# set columns names
names(plinkPCA)[1:ncol(plinkPCA)] <- paste0("PC", 1:(ncol(plinkPCA)))


# percentage variance explained
pve <- data.frame(PC = 1:20, pve = EigenValue/sum(EigenValue)*100)
#view(pve)

# PCA plot

# make plot
P <- ggplot(pve, aes(PC, pve)) + geom_bar(stat = "identity")
p1 <- P + ylab("Percentage variance explained") + theme_light()
ggsave("Percentage.variance.png", plot = p1, width = 8, height = 6, dpi = 300)


# plot pca
p2 <- ggplot(plinkPCA, aes(PC1, PC2)) + geom_point(size = 3)+ coord_equal() +
   theme_light()+
   coord_equal() +
   theme_light() + xlab(paste0("PC1 (", signif(pve$pve[1], 3), "%)")) + ylab(paste0("PC2 (", signif(pve$pve[2], 3), "%)"))

ggsave("PCA1.png", plot = p2, width = 8, height = 6, dpi = 300)

# lets divide population in two group
# (pop <- rep(c("A","B"), each=18))
#  pop <- as.data.frame(pop)
#  plinkPCA$pop <- pop$pop
 mypop = read_tsv("wild_captives.txt", col_names = F)
 
 plinkPCA$status <- mypop$X3
 plinkPCA$pop <- mypop$X5
 
p3 <- ggplot(plinkPCA, aes(PC1, PC2, color = as.factor(pop), shape = as.factor(status))) + geom_point(size = 3) + coord_equal() +
   geom_hline(yintercept = 0, linetype="dotted") + 
   geom_vline(xintercept = 0, linetype="dotted") +
   theme_light() + xlab(paste0("PC1 (", signif(pve$pve[1], 3), "%)")) + ylab(paste0("PC2 (", signif(pve$pve[2], 3), "%)"))+
   labs(color= "Group", shape = "Status") +
   ggtitle("PCA of BSF populations")
 
ggsave("PCA2.png", plot = p3, width = 8, height = 6, dpi = 300)


### ADMIXTURE

In [ ]:
#!/bin/bash

#more information: https://speciationgenomics.github.io/ADMIXTURE/
FILE="output"
VCF="path/to/vcffile"

#Generate the input file in plink format
plink-1.9-rc --vcf $VCF --make-bed --double-id --out $FILE --allow-extra-chr

#ADMIXTURE does not accept chromosome names that are not human chromosomes. We will thus just exchange the first column by 0
awk '{$1="0";print $0}' $FILE.bim > $FILE.bim.tmp
mv $FILE.bim.tmp $FILE.bim

#Run admixture with cross-validation (k=2..k=5)
for i in {2..11}
do
   admixture --cv $FILE.bed $i > log${i}.out
done

#To identify the best value of k clusters which is the value with lowest cross-validation error, we need to collect the cv errors.
awk '/CV/ {print $3,$4}' *out | cut -c 4,7-20 > $FILE.cv.error

awk '{split($1,name,"."); print $1,name[2]}' $FILE.nosex > $FILE.list


## Phylogenetic Analysis

In [ ]:
#!/bin/bash

VCF_File="path/to/vcf"

tassel-5.2.89-0/run_pipeline.pl -Xmx100G -SortGenotypeFilePlugin -inputFile "$VCF_File" -outputFile sorted.vcf -fileType VCF

tassel-5.2.89-0/run_pipeline.pl -Xmx100G -importGuess sorted.vcf -ExportPlugin -saveAs sequences.phy -format Phylip_Inter

iqtree -s sequences.phy -st DNA -m MFP -o PTA_000 -bb 1000 --bnni -cmax 15 -nt AUTO

## Additional Analyses

### Treemix

In [ ]:
#!/bin/bash

#more information: https://speciationgenomics.github.io/Treemix/

for i in {0..5}
do
 treemix -i BSF.noN.treemix.frq.gz -m $i -o BSF.$i -root OG -bootstrap -k 500 -noss  > treemix_${i}_log
done

### Genetic Diversity

In [ ]:
#!/bin/bash

#more information: https://github.com/simonhmartin/genomics_general
VCF="path/to/vcffile"

#Step 1: Nucleotide diversity
python3.7 genomics_general/parseVCF.py -i '$VCF' | bgzip --threads 8 > output.geno.gz

python2.7 genomics_general/popgenWindows.py \
    -g output.geno.gz \
    -o output.Fst.Dxy.pi.csv.gz \
    -T 8 \
    -f phased \
    -w 100000 \
    -m 100 \
    -s 100000 \
    $(awk '{print "-p", $1}' pops.txt) \
    --popsFile sample_pop.txt

#Step 2: FST and DXY
python2.7 genomics_general/popgenWindows.py \
    -g output.geno.gz \
    -o output.Fst.Dxy.pi.csv.gz \
    -T 8 \
    -f phased \
    -w 50000 \
    -m 50 \
    -s 50000 \
    $(awk '{print "-p", $1}' pops.txt) \
    --popsFile sample_pop.txt

#Step 3: Tajimas D
# Generate population-specific sample lists
while read pop; do
    grep -w "$pop" sample_pop.txt | cut -f1 > ${pop}.txt
done < pops.txt

# Step 3: Calculate Tajima's D for the given population
vcftools --gzvcf $VCF \
         --keep ${pop}.txt \
         --TajimaD 100000 \
         --out tajimasD_${pop}

### Heterozygosity/Inbreeding

In [ ]:
#!/bin/bash

#Step 1: Site frequency spectrum (SFS)
RENAME_FILE=$1   # Sample mapping file
ANGSD_OUT_DIR=$2 # Directory where ANGSD SAF outputs are stored
SFS_OUT_DIR=$3   # Directory to store estimated SFS outputs
REAL_SFS="angsd/misc/realSFS"  # Path to realSFS binary

# Ensure output directory exists
mkdir -p "$SFS_OUT_DIR"

# Check if arguments are provided
if [[ -z "$RENAME_FILE" || -z "$ANGSD_OUT_DIR" || -z "$SFS_OUT_DIR" ]]; then
    echo "Usage: $0 <sample_bam_mapping.txt> <angsd_output_dir> <sfs_output_dir>"
    exit 1
fi

# Get the total number of samples in the mapping file
TOTAL_SAMPLES=$(wc -l < "$RENAME_FILE")

# Ensure SLURM task ID does not exceed available samples
if [[ $SLURM_ARRAY_TASK_ID -gt $TOTAL_SAMPLES ]]; then
    echo "Error: Task ID exceeds the number of samples."
    exit 1
fi

# Extract sample name for this array job
SAMPLE=$(sed -n "${SLURM_ARRAY_TASK_ID}p" "$RENAME_FILE")

# Define input and output paths
SAF_FILE="${ANGSD_OUT_DIR}/${SAMPLE}_angsd.saf.idx"
SFS_FILE="${SFS_OUT_DIR}/${SAMPLE}_est.ml"

# Check if SAF file exists
if [[ ! -f "$SAF_FILE" ]]; then
    echo "Error: SAF file not found for sample $SAMPLE ($SAF_FILE)"
    exit 1
fi

# Run realSFS to Estimate SFS
echo "Estimating SFS for sample: $SAMPLE"
$REAL_SFS "$SAF_FILE" -P 1 > "$SFS_FILE"

# Check if SFS estimation was successful
if [[ ! -f "$SFS_FILE" ]]; then
    echo "Error: SFS file not created for sample $SAMPLE ($SFS_FILE)"
    exit 1
fi

#Step2: ANGSD heterozygosity
RENAME_FILE=$1   # Sample mapping file (sample_name <TAB> bam_file_path)
ANGSD_PATH="path/to/angsd"
REF_GENOME="path/to/ref"
ANGSD_OUT_DIR=$2 # Directory to store ANGSD outputs

# Ensure output directory exists
mkdir -p "$ANGSD_OUT_DIR"

# Check if arguments are provided
if [[ -z "$RENAME_FILE" || -z "$ANGSD_OUT_DIR" ]]; then
    echo "Usage: $0 <sample_bam_mapping.txt> <angsd_output_dir>"
    exit 1
fi

# Get the total number of samples in the mapping file
TOTAL_SAMPLES=$(wc -l < "$RENAME_FILE")

# Ensure SLURM task ID does not exceed available samples
if [[ $SLURM_ARRAY_TASK_ID -gt $TOTAL_SAMPLES ]]; then
    echo "Error: Task ID exceeds the number of samples."
    exit 1
fi

# Extract sample name and BAM path for this array job
SAMPLE_INFO=$(sed -n "${SLURM_ARRAY_TASK_ID}p" "$RENAME_FILE")
SAMPLE=$(echo "$SAMPLE_INFO" | awk '{print $1}')
BAM_PATH=$(echo "$SAMPLE_INFO" | awk '{print $2}')

# Check if BAM file exists
if [[ ! -f "$BAM_PATH" ]]; then
    echo "Error: BAM file for sample $SAMPLE not found ($BAM_PATH)"
    exit 1
fi

# Run ANGSD to Compute SAF
echo "Processing sample: $SAMPLE with BAM file: $BAM_PATH"
$ANGSD_PATH -i "$BAM_PATH" -anc "$REF_GENOME" -ref "$REF_GENOME" \
    -C 50 -minQ 20 -dosaf 1 -GL 1 -nThreads 4 -out "${ANGSD_OUT_DIR}/${SAMPLE}_angsd"



### ROH_FROH

In [ ]:
#!/bin/bash 

#Runs of homozygosity

VCF='path/to/vcffile'

# Convert VCF directly to BED format
plink-1.9-rc --vcf $VCF --make-bed --keep-allele-order --nonfounders --allow-no-sex --double-id --out output

#Run ROH Analysis in PLINK

#--homozyg: Enables ROH detection.
#--homozyg-snp 50: Minimum number of SNPs in an ROH.
#--homozyg-kb 1000: Minimum ROH length (1 Mb).
#--homozyg-density 50: Max density (1 SNP per 50 kb).
#--homozyg-gap 100: Max gap between SNPs in an ROH.

for pop in $(cat pops.txt); do
    echo "Processing ROH for population: $pop"
    plink-1.9-rc --bfile output --keep <(awk -v p="$pop" '$2==p {print $1, $1}' sample_pop.txt) \
          --homozyg --homozyg-window-het 3 --homozyg-window-missing 20 \
          --out "${pop}_roh" &
done

### Relatedness

In [ ]:
#!/bin/bash 

VCF='path/to/vcffile'

#Relatedness
plink-1.9-rc --vcf $VCF --make-bed --out output
king -b output.bed --related

### LD and Effective Population Size

In [ ]:
#!/bin/bash 

VCF='path/to/vcffile'

#Linkage Disequilibrium (LD)
PopLDdecay -InVCF $VCF -OutStat POP -MaxDist 500 -MAF 0.05

#Effective population size (Ne)
SNeP1.1 -ped output.ped -map output.map -phased -out output_Ne -threads 8 

## 📊 Expected Outputs
- Filtered BAM and VCF files
- PCA and ADMIXTURE plots
- Phylogenetic tree files
- Genetic diversity summary statistics
- ROH and relatedness plots

## 📌 Tips
- Adapt file paths and script names to your directory structure.
- Check for errors and inspect log files after each step.
- Keep track of software versions for reproducibility.



📫 **Contact**

For questions, suggestions, or contributions, please contact: \
[Peter Muchina] — [kimanimuchina@gmail.com]